## Bandpass Filter on IMERG data

## Import modules

In [29]:
import h5py
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

## Load data

In [30]:
# File Path
FILE_PATH = "/work/b11209013/2025_Research/IMERG/IMERG_06_17_lowpass.nc"

with xr.open_dataset(FILE_PATH) as f:
    
    f = f.sel(lat=slice(-10, 10)).mean(dim='lat')

    coords = f.coords

    prec = f["precipitation"]*24 # convert from mm/hr to mm/day

prec = prec - prec.mean(dim={'time', "lon"})

prec= prec.values

nt, nx = prec.shape

## Apply Bandpass Filter

### Define Functions

In [31]:
def fft2(data):
    data_fft = np.fft.fft(data, axis=0)
    data_fft = np.fft.ifft(data_fft, axis=1)

    return data_fft

def ifft2(data_fft):
    data = np.fft.ifft(data_fft, axis=0)
    data = np.fft.fft(data, axis=1)

    return data.real

### Setup Filter

In [32]:
# setup wavenumber and frequency
kn = np.fft.fftfreq(nx, d=1/nx)
fn = np.fft.fftfreq(nt, d=1)

kk, ff = np.meshgrid(kn, fn)

# setup filter
kel_cond = lambda k, ed: 86400.0/(2*np.pi*6.371e6)*k*np.sqrt(9.81*ed)

# design filter
k_domains = [
    (2*i+1, 2*i+3) for i in range(6)
]

mask = {}

for k in k_domains:
    mask[f"kw_{k[0]}_{k[1]}"] = np.where(
    (
        (kk >= k[0]) & (kk <= k[1]) &
        (ff >= 1/20) & (ff <= 1/2.5) &                     # KW band
        (ff >= kel_cond(kk, 8)) & (ff <= kel_cond(kk, 50)) # dispersion relation for KW
    ) |
    (
        (kk <= -k[0]) & (kk >= -k[1]) &
        (ff <= -1/20) & (ff >= -1/2.5) &                   # KW band
        (ff <= kel_cond(kk, 8)) & (ff >= kel_cond(kk, 50)) # dispersion relation for KW
    ), 1, 0
)

mask["mjo_1_4"] = np.where(
    (
        (kk >= 1) & (kk <= 4) &
        (ff >= 1/90) & (ff <= 1/30)
    ) |
    (
        (kk <= -1) & (kk >= -4) &
        (ff <= -1/90) & (ff >= -1/30)
    ), 1, 0
)

### Apply Filter

In [33]:
# apply fft on precipitation
prec_fft = fft2(prec)

# apply mask
prec_reconstructed = {
    key: ifft2(prec_fft * mask[key]) for key in mask.keys()
}


## Save File

In [35]:
PATH_OUTPUT = "/home/b11209013/2025_Research/Obs/Files/IMERG/"

with h5py.File(f"{PATH_OUTPUT}Hovmoller.h5", "w") as f:
    f.create_dataset("lon", data=coords["lon"].values)

    hov_grp = f.create_group("precip")

    for key in prec_reconstructed.keys():
        hov_grp.create_dataset(key, data=prec_reconstructed[key])
